In [ ]:
from github import Github
import itertools
import json
import os
import requests

#### Read personal token to authenticate.

In [ ]:
with open(os.path.expanduser('~') + '/.github_shell_token', 'r') as token_file:
    access_token = token_file.read().strip()
if access_token == None:
    error("Error: no access token found or provided")
g = Github(access_token)

In [ ]:
g.rate_limiting

#### These variables are global variables.

In [ ]:
repo_var = None
page_num=None
repo_list=list()
list_without_lesson=list()
list_with_lesson=list()
final_repo_list=list()

#### Declaring a function
* To get the total repositories count
* Creating a page_num variable to hold the valid page numbers to iterate the repo list.
* Final repo list created from repo list
* List with lesson created from final repo list
* List without lesson created from final repo list

In [ ]:
# Declaring the function.

def repo_initialize():
    
    '''
    Function takes no parameter.
    Returning total repositories count, final repository list, list_with_lesson & list_without_lesson.
    
    '''
    
    global repo_var
    global page_num
    global repo_list
    global list_without_lesson
    global list_with_lesson
    global final_repo_list
    
    repo_var = g.get_organization("carpentries-incubator").get_repos()

    page_num = 0
    for i in range(0,10):
        if len(repo_var.get_page(i))==0:
            page_num = i
            break
        else:
            pass
    repo_list = list()
    for i in range(0,page_num):
        repo_list.append(repo_var.get_page(i))
    final_repo_list = list((itertools.chain.from_iterable(repo_list)))

# Repository lists. 1)list_with_lesson & list_without_lesson.

    for repo in final_repo_list:
        tags=repo.get_topics()
        if "lesson" in tags:
            list_with_lesson.append(repo)
        else:
            list_without_lesson.append(repo)
            
# Calling the function

repo_initialize()

#### This approach is followed to retrieve data of each repo in seperate files that would encompass commits, PRs, issues, stages & labels. 

In [ ]:
HEADERS = {'Authorization': 'token ' + access_token}

#### Declaring a function to get the repositories by name.

In [ ]:
def get_by_name(name):
    '''
    Functions takes repo name as a parameter
    Returning the repo.
    '''
    for repo in list_with_lesson:
        if repo.name== name:
            return repo
repo=get_by_name('shell-extras')
    

#### Declaring functions to retrieve commits & contributors and Issues & Contributors and Pull request & Contributors Life cycle stages and Labels 

In [ ]:
#### Declaring a function to get the commit authors and commiters 


def commit_user_details(c):


    # first we try to retrieve details via PyGithub. This costs less API calls.
    try: 
        author = c.author.name
        committer = c.committer.name
        # The above case will raise an exception if c.author or c.committer is None.
        # This will be the case when the user does not have a GitHub account.
        # There may be another special case, when a user has a GitHub user name, but
        # does not specify their name. We need to detect it as well.
        if author == None or committer == None:
            raise Exception()
        return [author,committer]
    except:
        # If at least one of the names is not known, use GitHub API via `requests` library.
        # This will be slower and will cost some API calls, but at least we are spending
        # one API call per each commit instead of two.
        # Based on solution for https://github.com/jbon/Github-Social-Network-Analysis/issues/15
        commit_json = requests.get(c.url,headers=HEADERS).json()["commit"]
        author = commit_json['author']['name']
        committer = commit_json['committer']['name']
        return [author,committer]

    
    
# Function to retrieve Commits authors and committers.

def get_commits(repo):
    commits=repo.get_commits()
    counter=0
    commits_dict=dict()
    for c in commits:
        commits_dict[c.sha] = commit_user_details(c)
    return commits_dict

# Calling the function

commits_value=get_commits(repo)
    


    
# Function to retrieve PRs and usernames .

def get_PRs(repo):
    pr_list=[]
    pr=repo.get_pulls(state='open')
    for p in pr:
        pr_list.append({'id':p.id,'pr_name':p.user.login})
    return pr_list

#Calling the function

PR_value=get_PRs(repo)




# Function to retrieve issues and user names

def get_issues(repo):
    issue_list=[]
    issues=repo.get_issues(state='open')
    for issue in issues:
        issue_list.append({'title': issue.title, 'user_name':issue.user.login})
    return issue_list
 
#Calling the function

Issues_value=get_issues(repo)



# Function to retrieve life cycle stages of the lessons.

def repo_stage(repo):
    topics=repo.get_topics()
    stage=None
    for stage in topics:
        if stage in ["pre-alpha", "alpha", "beta", "stable"]:
            break;
    return stage

#Calling the function

life_cycle_stage=repo_stage(repo)
        

# Function to retrieve Labels of the lessons 

def repo_labels(repo):
    label_list=[]
    labels=repo.get_labels()
    for label in labels:
         label_list.append(label.name)
    return label_list

#Calling the function

labels_info=repo_labels(repo)
            
 




    


####  Store the result into a JSON file.

In [ ]:
with open('data/'+repo.name+'.json', 'w', encoding='utf-8') as f: 
        json.dump({'commits_dict':commits_value,'pr_dict':PR_value, 'issue_dict':Issues_value,'stage':life_cycle_stage,'label':labels_info}, f, ensure_ascii=False, indent=4) 